# Customizing transpilation

## Working with Preset Pass Managers

Qiskit includes functions to build preset [`PassManager`](/api/qiskit/qiskit.transpiler.PassManager) objects.
These preset passmanagers are used by the [`transpile`](/api/qiskit/qiskit.utils.QuantumInstance#transpile) function
for each optimization level. There are 4 optimization levels ranging from 0 to 3, where higher
optimization levels take more time and computational effort but may yield a
more optimal circuit.
Optimization level 0 is intended for device characterization experiments and, as such, only
maps the input circuit to the constraints of the target backend, without
performing any optimizations. Optimization level 3 spends the most effort to optimize the circuit.
However, as many of the optimization techniques in the transpiler are heuristic based, spending more
computational effort does not always result in an improvement in the quality of the output
circuit.

If you'd like to work directly with a
preset pass manager you can use the [`generate_preset_pass_manager`](/api/qiskit/transpiler_preset#qiskit.transpiler.preset_passmanagers.generate_preset_pass_manager)
function to easily generate one. For example, the following code generates a [`StagedPassManager`](/api/qiskit/qiskit.transpiler.StagedPassManager) object for optimization level 3
targeting the [`FakeLagosV2`](/api/qiskit/qiskit.providers.fake_provider.FakeLagosV2) backend (equivalent to what is used internally
by `transpile` with `backend=FakeLagosV2()` and `optimization_level=3`).

In [1]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeLagosV2

backend = FakeLagosV2()
pass_manager = generate_preset_pass_manager(3, backend)

You can use this just like you would any other `PassManager`. However,
because it is a `StagedPassManager` it also makes it easy to compose and/or
replace stages of the pipeline. For example, if you wanted to run a custom scheduling
stage using dynamical decoupling (via the [`PadDynamicalDecoupling`](https://docs.quantum-computing.ibm.com/api/qiskit-ibm-provider/qiskit_ibm_provider.transpiler.passes.scheduling.PadDynamicalDecoupling) pass) and
also add initial logical optimization prior to routing, you would do something like this:

In [3]:
import numpy as np
from qiskit.circuit.library import HGate, PhaseGate, RXGate, TdgGate, TGate, XGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (
    ALAPScheduleAnalysis,
    CXCancellation,
    InverseCancellation,
    PadDynamicalDecoupling,
)

backend_durations = backend.target.durations()
dd_sequence = [XGate(), XGate()]
scheduling_pm = PassManager(
    [
        ALAPScheduleAnalysis(backend_durations),
        PadDynamicalDecoupling(backend_durations, dd_sequence),
    ]
)
inverse_gate_list = [
    HGate(),
    (RXGate(np.pi / 4), RXGate(-np.pi / 4)),
    (PhaseGate(np.pi / 4), PhaseGate(-np.pi / 4)),
    (TGate(), TdgGate()),
]
logical_opt = PassManager(
    [
        CXCancellation(),
        InverseCancellation(inverse_gate_list),
    ]
)


# Add pre-layout stage to run extra logical optimization
pass_manager.pre_layout = logical_opt
# Set scheduling stage to custom pass manager
pass_manager.scheduling = scheduling_pm

Now, when the staged pass manager is run via the [`StagedPassManager.run`](/api/qiskit/qiskit.transpiler.StagedPassManager#run) method, the `logical_opt` pass manager will be called before the `layout` stage, and the `scheduling_pm` pass manager will be used for the `scheduling` stage instead of the default.

## Custom Pass Managers

In addition to modifying preset pass managers, it is also possible to construct a pass manager to build an entirely custom pipeline for transforming input circuits. You can use the [`StagedPassManager`](/api/qiskit/qiskit.transpiler.StagedPassManager) class directly to do this. You can define arbitrary stage names and populate them with a [`PassManager`](/api/qiskit/qiskit.transpiler.PassManager) instance. For example, the following code creates a new `StagedPassManager` that has 2 stages, `init` and `translation`.

In [19]:
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.passes import (
    Collect2qBlocks,
    ConsolidateBlocks,
    UnitarySynthesis,
    Unroll3qOrMore,
)

basis_gates = ["rx", "ry", "rxx"]
init = PassManager([UnitarySynthesis(basis_gates, min_qubits=3), Unroll3qOrMore()])
translate = PassManager(
    [
        Collect2qBlocks(),
        ConsolidateBlocks(basis_gates=basis_gates),
        UnitarySynthesis(basis_gates),
    ]
)

staged_pm = StagedPassManager(
    stages=["init", "translation"], init=init, translation=translate
)

There is no limit on the number of stages you can put in a `StagedPassManager`.

The [stage generator functions](/api/qiskit/transpiler_preset#stage-generator-functions) functions may be useful for the construction of custom pass managers.
They generate stages which provide common functionality used in many pass managers.
For example, [`generate_embed_passmanager`](/api/qiskit/transpiler_preset#qiskit.transpiler.preset_passmanagers.common.generate_embed_passmanager) can be used to generate a stage
to "embed" a selected initial `Layout` from a layout pass to the specified target device.